**Importing Libraries**

In [ ]:
import tensorflow
from tensorflow import keras
from keras.layers import Dense, Input
from keras.models import Sequential,Model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
import pickle
import numpy as np

**Loading and Preprocessing Data**

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

X_train =X_train/ 255
X_test =X_test/ 255

Y_train = tensorflow.keras.utils.to_categorical(Y_train, 10)
teststarget = Y_test
Y_test = tensorflow.keras.utils.to_categorical(Y_test, 10)

print("X_train: ",X_train.shape)
print("Y_train: ",Y_train.shape)
print("X_test: ",X_test.shape)
print("Y_test: ",Y_test.shape)

X_train:  (60000, 784)
Y_train:  (60000, 10)
X_test:  (10000, 784)
Y_test:  (10000, 10)


**Stack Autoencoder**

In [ ]:
inp1=Input(shape=(784,))
h1=Dense(100, activation='sigmoid')(inp1)
o1=Dense(784,activation='sigmoid')(h1)
AE1=Model(inputs=inp1,outputs=o1)
AE1.compile(optimizer='adam',metrics=['accuracy'],loss='mse')

print('Training the first AE')
AE1.fit(X_train,X_train,epochs=10)

hid_o1=AE1.layers[1].output
trim_o1=Model(inputs=inp1,outputs=hid_o1)
trim_o1_train=trim_o1.predict(X_train)
trim_o1_test=trim_o1.predict(X_test)

print('Training the Second AE')
inp2=Input(shape=(100,))
h2=Dense(50, activation='sigmoid')(inp2)
o2=Dense(100,activation='sigmoid')(h2)
AE2=Model(inputs=inp2,outputs=o2)
AE2.compile(loss='mse',metrics=['accuracy'],optimizer='adam')
print('Shape of AE2 layer1: ',AE2.layers[1].get_weights()[0].shape)
print('Shape of AE2 layer2: ',AE2.layers[2].get_weights()[0].shape)
AE2.fit(trim_o1_train,trim_o1_train,epochs=10)

hid_o2=AE2.layers[1].output
trim_o2=Model(inputs=inp2,outputs=hid_o2)
trim_o2_train=trim_o2.predict(trim_o1_train)
trim_o2_test=trim_o2.predict(trim_o1_test)

print('Training the Third AE(Not AE, but for classification)')
inp3=Input(shape=(50,))
o3=Dense(10, activation='softmax')(inp3)
AE3=Model(inputs=inp3,outputs=o3)
AE3.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
print('Shape of AE3 layer1: ',AE3.layers[1].get_weights()[0].shape)
AE3.fit(trim_o2_train,Y_train,epochs=10)


Training the first AE
Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0605 - accuracy: 0.0118
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0220 - accuracy: 0.0106
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0137 - accuracy: 0.0125
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0102 - accuracy: 0.0125
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0083 - accuracy: 0.0126
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0068 - accuracy: 0.0132
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0056 - accuracy: 0.0137
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0046 - accuracy: 0.0133
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0040 - accuracy: 0.0141
Epoch 10/10
1875/1875 [==============================] - 5s 

**Combining AE's and developing Classification model**

In [ ]:
AE=Sequential()
AE.add(AE1.layers[0])
AE.add(AE1.layers[1])
AE.add(AE2.layers[1])
AE.add(AE3.layers[-1])
AE.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
AE.fit(X_train,Y_train,epochs=5)

predic = np.array(AE.predict(X_test))
prediction = np.argmax(predic, axis=1)
print("Accuracy score of classification is: ", accuracy_score(teststarget, prediction))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3362 - accuracy: 0.9016
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1831 - accuracy: 0.9473
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1294 - accuracy: 0.9618
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0989 - accuracy: 0.9720
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0756 - accuracy: 0.9774
Accuracy score of classification is:  0.9696
